In [4]:
#Import
import matplotlib.pyplot as plt
from problem_formulation import get_model_for_problem_formulation
from ema_workbench import (ema_logging, MultiprocessingEvaluator, Scenario, ScalarOutcome)
from ema_workbench.em_framework.optimization import (ArchiveLogger,EpsilonProgress)
#from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator


ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [5]:
#Import function
model, planning_steps = get_model_for_problem_formulation(2)

#Outcomes
model.outcomes = [ScalarOutcome("Expected Annual Damage",kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome("Dike Investment Costs",kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome("RfR Investment Costs", kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome("Evacuation Costs", kind=ScalarOutcome.MINIMIZE),
            ScalarOutcome("Expected Number of Deaths",kind=ScalarOutcome.MINIMIZE)]

#Reference scenario - conservative approach with "worst-case" behaviour.
reference_scenario = Scenario('reference', **{'discount rate 0': 1.5, 'discount rate 1': 1.5, 'discount rate 2': 1.5, 'A.0_ID flood wave shape': 75, 'A.1_Bmax': 240, 'A.1_pfail': 0.25, 'A.1_Brate': 10, 'A.2_Bmax': 240, 'A.2_pfail': 0.25, 'A.2_Brate': 10, 'A.3_Bmax': 240, 'A.3_pfail': 0.25, 'A.3_Brate': 10, 'A.4_Bmax': 240, 'A.4_pfail': 0.25, 'A.4_Brate': 10, 'A.5_Bmax': 240, 'A.5_pfail': 0.25, 'A.5_Brate': 10})

epsilon = [0.1,]*len(model.outcomes)

In [6]:
convergence_metrics = [EpsilonProgress()]

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=1e3, searchover='levers',
                                 convergence=convergence_metrics,
                                 epsilons= epsilon, reference=reference_scenario)


  0%|                                                 | 0/1000 [00:00<?, ?it/s]Process SpawnPoolWorker-18:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/juanita/Documents/GitHub/epa1361_open/Py310-MBDM/lib/python3.11/site-packages/ema_workbench/em_framework/ema_multiprocessing.py", line 168, in worker
    return experiment, experiment_runner.run_experiment(experiment)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/juanita/Documents/GitHub/epa1361

KeyboardInterrupt: 

In [ ]:
from ema_workbench.analysis import parcoords

data = results.loc[:, [o.name for o in model.outcomes]]
limits = parcoords.get_limits(data)

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
plt.show()